In [1]:
import numpy as np
import pandas as pd
import json
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from tqdm import tqdm_notebook, trange



import gensim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import functions 
clean = functions.clean()

In [2]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/{}.json"

with open(path.format('train_test_split_update'), 'r') as readfile:
    train_test_split = json.load(readfile)
    readfile.close()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/train_test_split_update.json'

In [3]:
#Open cases
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/{}.json"

with open(path.format('text_none'), 'r') as readfile:
    text_none = json.load(readfile)
    readfile.close()
    
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/{}.json"

with open(path.format('text_english'), 'r') as readfile:
    text_english = json.load(readfile)
    readfile.close()
    


FileNotFoundError: [Errno 2] No such file or directory: '/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/text_none.json'

In [5]:
list(text_english.keys())[1]

'HUDOC-ECHR-1982-001-57580'

In [5]:
# Fucntions
def getPart(ID,part,stopwords):
    """
    Returns clean case part
    """
    
    if stopwords == 'none':
        doc = text_none[ID][part]
    elif stopwords == 'english':
        doc = text_english[ID][part]
    return doc


def getCorpus(article,part,stopwords):
    """
    returns the train test corpus along with target
    """
    split = train_test_split[article]
    
    train_corpus = []
    test_corpus = []
    
    for ID in split['v_train'] + split['nv_train']:
        doc = getPart(ID,part,stopwords)
        train_corpus.append(doc)
        
    for ID in split['v_test']+ split['nv_test']:
        doc = getPart(ID,part,stopwords)
        test_corpus.append(doc)
        
    train_target = [1]*len(split['v_train']) + [0]*len(split['nv_train'])
    test_target = [1]*len(split['v_test']) + [0]*len(split['nv_test'])
    
    
    return train_corpus, test_corpus, train_target, test_target
    
    
train_corpus, test_corpus, train_target, test_target = getCorpus('6','procedure','none')
print(len(train_corpus),len(test_corpus),len(train_target),sum(train_target),len(test_target),sum(test_target))



1008 595 1008 504 595 539


## N-grams

In [13]:
def ngram(article,part,stopwords):
    """
    Create ngram features for article and hyper parameters
    """
    scaler = preprocessing.MinMaxScaler()
    max_features = 2000
    ngram_range = (1,4)
    
    train_corpus, test_corpus, train_target, test_target = getCorpus(article,part,stopwords)
    
    vectorizer = CountVectorizer(max_features= max_features,
                                 ngram_range=ngram_range)
    #Train features
    train = vectorizer.fit_transform(train_corpus,y=train_target).toarray()
    train = scaler.fit_transform(train)
    df_train = pd.DataFrame(data = train, columns=vectorizer.get_feature_names())
    df_train['target'] = train_target
    
    #Test features
    test = vectorizer.transform(test_corpus).toarray() #check what what y means
    test = scaler.transform(test)
    df_test = pd.DataFrame(data = test, columns=vectorizer.get_feature_names())
    df_test['target'] = test_target
    
    return df_train, df_test

df_train, df_test = ngram('11','procedure','none')
df_train.head()

,accordance,accordance with,accordance with rule,accordance with the,addresses,addresses by,addresses by mr,admissibility,admissibility and,admissibility and merits,...,would represent him rule,would submit,would submit his,would submit his observations,written,written observations,written procedure,written procedure article,written procedure article of,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
df_train.values

In [ ]:
 'ngram_2000_9_circumstances_english',
file = test[6]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],ngram)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])

In [15]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/test_features/{}.csv"


filename = "{}_ngram_{}_{}_{}_{}"
train_filename = filename.format("train","2000","9","circumstances","english")
test_filename = filename.format("test","2000","9","circumstances","english")

df_train, df_test = ngram("9","circumstances","english")

df_train.to_csv(path.format(train_filename),index=False)
df_test.to_csv(path.format(test_filename),index=False)

In [10]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/features2/{}.csv"

for article in ["7","9","11","18"]:
    for part in ['both',"procedure","facts","circumstances","relevant"]:
        for stopwords in ['none','english']:
            
            filename = "{}_ngram_{}_{}_{}_{}"
            train_filename = filename.format("train","2000",article,part,stopwords)
            test_filename = filename.format("test","2000",article,part,stopwords)
            
            df_train, df_test = ngram(article,part,stopwords)
            
            df_train.to_csv(path.format(train_filename),index=False)
            df_test.to_csv(path.format(test_filename),index=False)
    print(article)

7
9
11
18


## Average embeddings

In [7]:
# Load embeddings
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/embeddings/{}"
echr_100 = Word2Vec.load(path.format('echt2vec_100.txt'))
echr_200 = Word2Vec.load(path.format('echt2vec_200.txt'))
print("ECHR")

law_100 = gensim.models.KeyedVectors.load_word2vec_format(path.format('Law2Vec.100d.txt'), binary=False)
law_200 = gensim.models.KeyedVectors.load_word2vec_format(path.format('Law2Vec.200d.txt'), binary=False)
print("LAW")
word_100 = dict()
f = open(path.format('glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_100[word] = coefs
f.close()


word_200 = dict()
f = open(path.format('glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_200[word] = coefs
f.close()
print("GLOVE")

/Users/conorosully/miniconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


ECHR
LAW
GLOVE


In [8]:
def averageVectors(corpus,embeddings):
    """
    returns and average embedding feature set 
    """
    
    docs_vectors = pd.DataFrame() # creating empty final dataframe
    #stopwords = nltk.corpus.stopwords.words('english') # removing stop words
    for doc in corpus:
        
        series = pd.Series(doc.split(' ')).value_counts()
        labels = series.index
        counts = series.values
        denom = sum(counts)
        
        temp = []
        for word,count in zip(labels,counts): # looping through each word of a single document and spliting through space
            try:
                word_vec = embeddings[word]*count # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp.append(word_vec) # if word is present then append it to temporary dataframe
            except:
                pass
        doc_vector = sum(temp)/denom 

        docs_vectors = docs_vectors.append(pd.Series(doc_vector), ignore_index = True) # append each document value to the final dataframe
    
        
    return docs_vectors

def averageEmbedding(article,part,stopwords,embeddings):
    """
    Return the average embedding features sets
    """
    
    scaler = preprocessing.MinMaxScaler()
    train_corpus, test_corpus, train_target, test_target = getCorpus(article,part,stopwords)

    train = averageVectors(train_corpus,embeddings).fillna(0).values
    train = scaler.fit_transform(train)
    df_train = pd.DataFrame(data = train)
    df_train['target'] = train_target
    
    #Test features
    test = averageVectors(test_corpus,embeddings).fillna(0).values
    test = scaler.transform(test)
    df_test = pd.DataFrame(data = test)
    df_test['target'] = test_target
    
    return df_train, df_test

df_train, df_test = averageEmbedding('2','both','none',word_100)
print('done')

done


In [9]:
def saveWordEmbeddings(df_train,df_test, filename, article,part,stopwords):
    train_filename = filename.format("train",article,part,stopwords)
    test_filename = filename.format("test",article,part,stopwords)
    
    df_train.to_csv(path.format(train_filename),index=False)
    df_test.to_csv(path.format(test_filename),index=False)
    

In [11]:
test = ['law_100_2_circumstances_english',
 'glove_200_3_both_none',
 'glove_200_5_relevant_english',
 'echr_100_6_both_english',
 'glove_200_7_circumstances_none',
 'echr_100_8_both_english',
 'ngram_2000_9_circumstances_english',
 'echr_200_10_both_none',
 'glove_200_11_both_english',
 'glove_100_13_both_english',
 'echr_200_14_both_none',
 'echr_200_18_both_none']

In [17]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/test_features/{}.csv"

file = test[0]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],law_100)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])


file = test[1]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],word_200)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])

file = test[2]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],word_200)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])

file = test[3]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],echr_100)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])

file = test[4]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],word_200)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])


file = test[5]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],echr_100)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])



file = test[7]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],echr_200)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])


file = test[8]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],word_200)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])


file = test[9]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],word_100)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])

 
file = test[10]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],echr_200)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])


file = test[11]
filename = "{}_" + file
split = file.split("_")
df_train, df_test = averageEmbedding(split[2],split[3],split[4],echr_200)
print(len(df_train),len(df_test))
saveWordEmbeddings(df_train, df_test,filename,split[2],split[3],split[4])


152 66
490 202
332 186


/Users/conorosully/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


1008 595
64 9
542 123
256 59
46 17
202 149
364 40
26 3


In [16]:
for article in tqdm_notebook(["7","9","11","18"]):
    for part in ['both',"procedure","facts","circumstances","relevant"]:
        for stopwords in ['none','english']:
            
            #glove 100
            filename = "{}_glove_100_{}_{}_{}"
            df_train, df_test = averageEmbedding(article,part,stopwords,word_100)
            saveWordEmbeddings(df_train, df_test,filename,article,part,stopwords)
            #glove 200
            filename = "{}_glove_200_{}_{}_{}"
            df_train, df_test = averageEmbedding(article,part,stopwords,word_200)
            saveWordEmbeddings(df_train, df_test,filename,article,part,stopwords)
            

            #law 100
            filename = "{}_law_100_{}_{}_{}"
            df_train, df_test = averageEmbedding(article,part,stopwords,law_100)
            saveWordEmbeddings(df_train, df_test,filename,article,part,stopwords)
            #law 200
            filename = "{}_law_200_{}_{}_{}"
            df_train, df_test = averageEmbedding(article,part,stopwords,law_200)
            saveWordEmbeddings(df_train, df_test,filename,article,part,stopwords)

            
            #echr 100
            filename = "{}_echr_100_{}_{}_{}"
            df_train, df_test = averageEmbedding(article,part,stopwords,echr_100)
            saveWordEmbeddings(df_train, df_test,filename,article,part,stopwords)
            #echr 200
            filename = "{}_echr_200_{}_{}_{}"
            df_train, df_test = averageEmbedding(article,part,stopwords,echr_200)
            saveWordEmbeddings(df_train, df_test,filename,article,part,stopwords)
            
            #doc 100
            filename = "{}_doc_100_{}_{}_{}"
            df_train, df_test = docEmbedding(article,part,stopwords,doc_100)
            saveWordEmbeddings(df_train, df_test,filename,article,part,stopwords)
            #doc 200
            filename = "{}_doc_200_{}_{}_{}"
            df_train, df_test = docEmbedding(article,part,stopwords,doc_200)
            saveWordEmbeddings(df_train, df_test,filename,article,part,stopwords)
            
    

/Users/conorosully/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
df_test.head()

## Doc2vec embeddings

In [14]:
# LoadModels
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/embeddings/{}"
doc_100 = Doc2Vec.load(path.format('doc2vec_100')) 
doc_200 = Doc2Vec.load(path.format('doc2vec_200')) 

In [15]:
def docVectors(corpus,model):
    """
    returns and average embedding feature set 
    """
    
    docs_vectors = pd.DataFrame() # creating empty final dataframe
    for doc in corpus:
       
        doc_vector = pd.Series(model.infer_vector(doc,steps = 20))
        
        docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
    
    return docs_vectors
    
def docEmbedding(article,part,stopwords,model):
    """
    Return the average embedding features sets
    """
    
    scaler = preprocessing.MinMaxScaler()
    train_corpus, test_corpus, train_target, test_target = getCorpus(article,part,stopwords)

    train = docVectors(train_corpus,model).fillna(0).values
    train = scaler.fit_transform(train)
    df_train = pd.DataFrame(data = train)
    df_train['target'] = train_target
    
    #Test features
    test = docVectors(test_corpus,model).fillna(0).values
    test = scaler.transform(test)
    df_test = pd.DataFrame(data = test)
    df_test['target'] = test_target
    
    return df_train, df_test


In [53]:
df_train, df_test = docEmbedding('2','procedure','english',doc_100)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,target
0,0.701853,0.288264,0.911129,0.817519,0.666821,0.617580,0.757744,0.362429,0.336497,0.626623,...,0.201242,0.292513,0.140923,0.773359,0.810436,0.500381,0.120092,0.199455,0.194301,1
1,0.871552,0.038642,0.487415,0.991884,0.714283,0.772244,0.857719,1.000000,0.249762,0.770412,...,0.216814,0.108396,0.169406,0.764955,0.768360,0.028876,0.228581,0.077343,0.276292,1
2,0.741251,0.194626,0.733237,0.806997,0.723123,0.692224,0.858634,0.421790,0.157690,0.484320,...,0.185298,0.182500,0.167108,0.686322,0.697989,0.579541,0.176833,0.212538,0.121831,1
3,0.812235,0.215559,0.929595,0.848726,0.762768,0.849760,0.675506,0.600225,0.284654,0.984758,...,0.198924,0.232102,0.409667,0.799542,0.714503,0.518365,0.208994,0.087479,0.494510,1
4,0.809568,0.309279,0.455995,0.746146,0.633901,0.597871,0.706190,0.376005,0.202541,0.642230,...,0.257492,0.347788,0.194931,0.699961,0.822112,0.651407,0.180120,0.299722,0.790947,1


### FEATURE SET LIST

In [17]:
features = []

path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/features2/{}.pickle"
with open(path.format('features'), 'wb') as f:

    for article in ["7","9","11","18"]:
        for part in ['both',"procedure","facts","circumstances","relevant"]:
            for stopwords in ['none','english']:
            
                filename = "{}_{}_{}_{}_{}"
                
                features.append(filename.format("ngram","2000",article,part,stopwords))
                
                features.append(filename.format("glove","100",article,part,stopwords))
                features.append(filename.format("glove","200",article,part,stopwords))

                features.append(filename.format("law","100",article,part,stopwords))
                features.append(filename.format("law","200",article,part,stopwords))
                
                features.append(filename.format("echr","100",article,part,stopwords))
                features.append(filename.format("echr","200",article,part,stopwords))
                
                features.append(filename.format("doc","100",article,part,stopwords))
                features.append(filename.format("doc","200",article,part,stopwords))
                
                
    pickle.dump(features,f)
    
    f.close()
print(len(features)  )        
                
                
                


360


In [ ]:
def averageVectorsOLD(corpus,embeddings):
    """
    returns and average embedding feature set 
    """
    
    docs_vectors = pd.DataFrame() # creating empty final dataframe
    #stopwords = nltk.corpus.stopwords.words('english') # removing stop words
    for doc in corpus:
        temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
        for word in doc.split(' '): # looping through each word of a single document and spliting through space
            try:
                word_vec = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
        doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
        docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe

        
    return docs_vectors